In [6]:
import numpy as np
import cvxpy as cp

# Parameters
no_of_sources = 2
no_of_slots = 2
snr = 15
available_power = 0.4
np.random.seed(14)

In [7]:
# Generate random fading coefficients and source signals
fading_coeff = np.random.randn(no_of_slots, no_of_sources)
print("H=",fading_coeff)
source = np.random.uniform(-1, 1, (no_of_sources, 1))  # Column vector
print("x=",source)
# Define optimization variable
pre_processing = cp.Variable((no_of_slots, no_of_sources))
print("B=",pre_processing)


# Correct Channel Matrix (use matrix multiplication, not element-wise multiplication)
channel_matrix = cp.multiply(fading_coeff , pre_processing ) # Now it's a valid linear operation
# channel_matrix=fading_coeff @ pre_processing
print("H^=",channel_matrix)


# Compute signal and noise parameters
signal_power = np.sum(source**2)
snr_linear = 10**(snr / 10.0)
noise_variance = signal_power / snr_linear
noise = noise_variance * np.eye(no_of_sources)  # Diagonal noise matrix
print("n=",noise)


H= [[ 1.55133911  0.07918602]
 [ 0.17397653 -0.07233657]]
x= [[ 0.02623342]
 [-0.36343115]]
B= var8
H^= [[1.55 0.08]
 [0.17 -0.07]] @ var8
n= [[0.00419857 0.        ]
 [0.         0.00419857]]


In [8]:
# Compute Post-Processing Matrix (Inverse-like operation)
regularized_matrix = signal_power * (channel_matrix @ channel_matrix.T) + noise
print("=",regularized_matrix)
print("Shape of regularized_matrix:", regularized_matrix.shape)



= Promote(0.13277039353913003, (2, 2)) @ [[1.55 0.08]
 [0.17 -0.07]] @ var8 @ [[1.55 0.08]
 [0.17 -0.07]] @ var8.T + [[0.00 0.00]
 [0.00 0.00]]
Shape of regularized_matrix: (2, 2)


In [9]:
# Instead of incorrect `cp.matrix_frac`, use `cp.inv_pos`
# post_processing = np.linalg.inv(regularized_matrix)@ channel_matrix @ np.ones((no_of_sources,1))  # Equivalent to pseudo-inverse operation
# print("a=",post_processing)


In [10]:
post_processing = cp.inv_pos(regularized_matrix)@ channel_matrix @ np.ones((no_of_sources,1))  # Equivalent to pseudo-inverse operation
print("a=",post_processing)

a= power(Promote(0.13277039353913003, (2, 2)) @ [[1.55 0.08]
 [0.17 -0.07]] @ var8 @ [[1.55 0.08]
 [0.17 -0.07]] @ var8.T + [[0.00 0.00]
 [0.00 0.00]], -1.0) @ [[1.55 0.08]
 [0.17 -0.07]] @ var8 @ [[1.00]
 [1.00]]


In [11]:

# Correct Cost Function (Ensure compatibility in matrix dimensions)
cost = cp.norm(post_processing.T @ (channel_matrix @ source + noise) - np.ones((1,no_of_sources))@ source     )

# Define Optimization Problem
objective = cp.Minimize(cost)

constraints = [
    cp.sum(pre_processing, axis=0) <= available_power
]

problem = cp.Problem(objective, constraints)
problem.solve()

# Print results
print("Optimal Pre-processing Matrix:")
print(pre_processing.value)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
[[1.55 0.08]
 [0.17 -0.07]] @ var8 @ [[1.55 0.08]
 [0.17 -0.07]] @ var8.T